In [1]:
import pandas as pd
import sys
import numpy as np
import json
from datetime import datetime, timedelta
sys.path.append("../helper_functions")
import defillama_utils as dfl
import duneapi_utils as du
import opstack_metadata_utils as ops
sys.path.pop()

current_utc_date = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
print(current_utc_date)

2024-01-22 00:00:00


In [2]:
chains = [
    #L2s
     ['Optimism', 'L2']
    ,['Base', 'L2']
    ,['Arbitrum', 'L2']
    #non-dune L2s
    ,['ZkSync Era', 'L2']
    ,['Polygon zkEVM', 'L2']
    ,['Starknet', 'L2']
    ,['Linea', 'L2']
    ,['Mantle', 'L2']
    ,['Scroll', 'L2']
    ,['Boba', 'L2']
    ,['Metis', 'L2']
    ,['opBNB', 'L2']
    ,['Rollux', 'L2']
    ,['Manta', 'L2']
    ,['Kroma','L2']
    ,['Arbitrum%20Nova','L2']
    #L1
    ,['Ethereum', 'L1']
    #Others
    ,['Fantom', 'L1']
    ,['Avalanche', 'L1']
    ,['Gnosis' , 'L1']
    ,['Celo', 'L1']
    ,['Polygon', 'L1']
    ,['BSC', 'L1']
]

protocols = [
         ['aevo', 'L2',['optimism','arbitrum','ethereum']]
        ,['dydx', 'L2',['ethereum']]
        ,['immutablex', 'L2',['ethereum']]
        ,['apex-protocol', 'L2',['ethereum']]
        # ,['brine.fi', 'L2',['ethereum']] #no longer listed
        ,['loopring', 'L2',['ethereum']]
        ,['aztec', 'L2',['ethereum']]
    ]

In [3]:
p_agg = []
for p in protocols:
    try:
        d = dfl.get_single_tvl(p[0], p[2])
        # print(f"Raw content for {p}: {d}")
        d['chain_prot'] = p[0].title()
        d['layer'] = p[1]
        d['defillama_slug'] = 'protocols/' + p[0]
        d['source'] = 'protocol'
        # d['prot_chain'] = c
        p_agg.append(d)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for {p}: {e}")
        continue  # Continue to the next iteration of the loop
    except Exception as e:
        print(f"An unexpected error occurred for {p}: {e}")
        continue  # Continue to the next iteration of the loop

df = pd.concat(p_agg)

df_sum = df.groupby(['date', 'chain_prot', 'layer', 'defillama_slug', 'source']).sum(['usd_value'])
df_sum.reset_index(inplace=True)

df_sum = df_sum[['date', 'usd_value', 'chain_prot', 'layer', 'defillama_slug', 'source']]
df_sum = df_sum.rename(columns={'chain_prot': 'chain', 'usd_value': 'tvl'})

print(df_sum.tail(5))


                    date           tvl          chain layer  \
4786 2024-01-22 13:38:59  3.559686e+07     Immutablex    L2   
4787 2024-01-22 13:41:59  1.077423e+08       Loopring    L2   
4788 2024-01-22 14:02:59  7.855720e+06          Aztec    L2   
4789 2024-01-22 14:04:59  4.902326e+07           Aevo    L2   
4790 2024-01-22 14:07:59  7.057174e+07  Apex-Protocol    L2   

               defillama_slug    source  
4786     protocols/immutablex  protocol  
4787       protocols/loopring  protocol  
4788          protocols/aztec  protocol  
4789           protocols/aevo  protocol  
4790  protocols/apex-protocol  protocol  


In [4]:
c_agg = []
for c in chains:
        d = dfl.get_historical_chain_tvl(c[0])
        d['chain'] = c[0]
        d['layer'] = c[1]
        d['defillama_slug'] = c[0]
        d['source'] = 'chain'
        c_agg.append(d)

df_ch = pd.concat(c_agg)
df = pd.concat([df_ch,df_sum])
# Rename
df['chain'] = df['chain'].str.replace('%20', ' ', regex=False)
df['chain'] = df['chain'].str.replace('-', ' ', regex=False)
df.loc[df['chain'] == 'Optimism', 'chain'] = 'OP Mainnet'
df.loc[df['chain'] == 'BSC', 'chain'] = 'BNB'
df.loc[df['chain'] == 'Brine.Fi', 'chain'] = 'Brine'
df.loc[df['chain'] == 'Immutablex', 'chain'] = 'ImmutableX'


df = df[df['date'] <= current_utc_date ] #rm dupes at current datetime

df['date'] = pd.to_datetime(df['date']) - timedelta(days=1) #map to the prior day, since dfl adds an extra day

In [5]:
# Add Metadata
chain_meta = pd.read_csv('../op_chains_tracking/inputs/chain_metadata.csv')
chain_meta = chain_meta.applymap(lambda x: x.strip() if isinstance(x, str) else x)

meta_cols = ['defillama_slug', 'is_op_chain','mainnet_chain_id','op_based_version']

df = df.merge(chain_meta[meta_cols], on='defillama_slug', how = 'left')

In [6]:
df = ops.generate_alignment_column(df)

In [7]:
# #Fill 
# df = df.fillna('')

In [8]:
print(df.sample(10))
# print(df[df['chain'] == 'Aevo'])

# print(df['chain'].unique())

            date           tvl       chain layer   defillama_slug    source  \
19824 2023-11-03  7.124228e+06       Aztec    L2  protocols/aztec  protocol   
13228 2021-01-09  3.766000e+05     Polygon    L1          Polygon     chain   
12233 2021-08-03  3.264468e+08        Celo    L1             Celo     chain   
9536  2023-10-10  4.651183e+07      Fantom    L1           Fantom     chain   
13775 2022-07-10  1.397888e+09     Polygon    L1          Polygon     chain   
12399 2022-01-16  7.335349e+08        Celo    L1             Celo     chain   
1948  2023-12-01  2.185253e+09    Arbitrum    L2         Arbitrum     chain   
9758  2021-05-31  2.102377e+08   Avalanche    L1        Avalanche     chain   
12385 2022-01-02  7.429772e+08        Celo    L1             Celo     chain   
2254  2023-01-20  0.000000e+00  ZkSync Era    L2       ZkSync Era     chain   

      is_op_chain mainnet_chain_id op_based_version      alignment  
19824         NaN              NaN              NaN  OP Stack

In [9]:
# Write to Dune
du.write_dune_api_from_pandas(df, 'dfl_chain_tvl',\
                             'TVL for select chains from DefiLlama')

table at: dune.oplabspbc.dataset_dfl_chain_tvl
Response status code: 200
Response content: b'{"success":true,"table_name":"dfl_chain_tvl"}'
